In [1]:
from huggingface_hub import login

login()

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, M2M100ForConditionalGeneration, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM, NllbTokenizerFast
from tokenization_small100 import SMALL100Tokenizer
from peft import LoraModel

W1121 22:24:10.447000 27584 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [4]:
from peft import LoraConfig, get_peft_model

FAIRSEQ_LANGUAGE_CODES = ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'pes_Arab', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'knc_Arab', 'knc_Latn', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kon_Latn', 'kor_Hang', 'kmr_Latn', 'lao_Laoo', 'lvs_Latn', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Latn', 'mkd_Cyrl', 'plt_Latn', 'mlt_Latn', 'mni_Beng', 'khk_Cyrl', 'mos_Latn', 'mri_Latn', 'zsm_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'gaz_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pol_Latn', 'por_Latn', 'prs_Arab', 'pbt_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Beng', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'als_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'taq_Latn', 'taq_Tfng', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zul_Latn']  # fmt: skip

tokenizer = NllbTokenizerFast.from_pretrained("facebook/nllb-200-distilled-600M",
                                                src_lang="tgl_Latn",
                                                tgt_lang="eng_Latn",)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", quantization_config=bnb_config)



In [5]:
for name, param in model.named_parameters():
    if "lora" in name.lower():
        print(name, param.requires_grad)

In [6]:
from peft import PeftModel

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "v_proj", "k_proj"],
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

peft_model = PeftModel.from_pretrained(model, "training-nllb-tgl-to-english-v2-working/checkpoint-6581")
peft_model.print_trainable_parameters()

trainable params: 0 || all params: 616,843,264 || trainable%: 0.0000


In [7]:
import pandas as pd
import datasets

tgl_eng = pd.read_csv("tagalog-to-english-corpora.csv")

def preprocess(batch):
    model_inputs = tokenizer(
        batch["language1_text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    labels = tokenizer(
        batch["language2_text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


eval_df = tgl_eng.sample(frac=0.075, random_state=67)
eval_dataset = datasets.Dataset.from_pandas(eval_df)
eval_dataset_processed = eval_dataset.map(preprocess, batched=True, remove_columns=['language1_text', 'language2_text'])

Map:   0%|          | 0/2189 [00:00<?, ? examples/s]

In [8]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="training-nllb-tgl-to-english-v2-working-testing",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=1,
    save_strategy="steps",
    save_steps=1,
    load_best_model_at_end=True,
    logging_strategy="steps",
    logging_steps=1,
    logging_first_step=True,
    report_to="none",
    predict_with_generate=True,
)

In [9]:
from transformers import Seq2SeqTrainer
import numpy as np
import evaluate

sacrebleu = evaluate.load("sacrebleu")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=None,
    eval_dataset=eval_dataset_processed,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\rek\AppData\Local\Temp\ipykernel_27584\2512045289.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [10]:
trainer.evaluate()

{'eval_loss': 6.4765625,
 'eval_model_preparation_time': 0.0036,
 'eval_bleu': 41.269,
 'eval_gen_len': 37.1549,
 'eval_runtime': 813.137,
 'eval_samples_per_second': 2.692,
 'eval_steps_per_second': 0.168}

In [16]:
def translate(text: str):
    tokens = tokenizer(text, return_tensors="pt").to(model.device)
    translated_tokens = peft_model.generate(
        **tokens, forced_bos_token_id=tokenizer.convert_tokens_to_ids("eng_Latn"), max_length=30,
    )
    text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    return text

In [22]:
#sentences = [
#    'Maaga akong nagising kanina.',
#    'Kumain kami ng almusal sa labas.',
#    'Malakas ang ulan buong maghapon.',
#    'Naglalakad ako papunta sa trabaho.',
#    'Masarap ang nilutong adobo ni mama.',
#    'Naghihintay ako ng sagot mula sa iyo.',
#    'Gusto kong magpahinga ngayong linggo.',
#    'Tinawagan ko siya pero hindi sumagot.',
#    'Napagod ako sa dami ng gawain.',
#    'Nanonood kami ng pelikula kagabi.',
#    'Nagulat ako sa balitang narinig ko.',
#    'Gusto kong matutong magluto ng ginataan.',
#    'Nakita ko siya sa silid-aralan kahapon.',
#    'Malapit na ang kaarawan ko.',
#    'Naglakbay kami papunta sa probinsya.',
#    'Mabait ang bagong kapitbahay namin.',
#    'Hinahanap ko ang susi ng kotse.',
#    'Naiwan ko ang wallet ko sa bahay.',
#    'Masaya ako para sa iyo.',
#    'Nag-aaral siya para sa pagsusulit bukas.',
#    'Uminom ako ng kape bago umalis.',
#    'Nalito ako sa direksyon na binigay mo.',
#    'Umuulan pero mainit ang panahon.',
#    'Naghintay kami nang halos isang oras.',
#    'Nagpadala ako ng pera kaninang umaga.',
#    'Gusto kong mag-aral ng bagong wika.',
#    'Nagpunta kami sa palengke para mamili.',
#    'Mas mabilis ngayon ang internet.',
#    'Umorder kami ng pagkain.',
#    'Sumakay ako ng bus papunta sa lungsod.',
#    'Nakakatuwa ang kwento niya.',
#    'Naghahanda kami para sa pagtitipon.',
#    'Nawalan kami ng kuryente kagabi.',
#    'Mataas ang marka niya sa pagsusulit.',
#    'Nagpapahinga lang ako sandali.',
#    'Tumawag ako para magpakonsulto.',
#    'Maganda ang tanawin sa bundok.',
#    'Naglalaro ang mga bata sa labas.',
#    'Mabango ang bagong sabon.',
#    'Nagpipinta siya ng larawan sa sala.',
#    'Nakikinig ako ng musika habang nagtratrabaho.',
#    'Dumating na ang inorder kong pagkain.',
#    'Nais ko pang matuto ng maraming bagay.',
#    'Naglalakad kami sa tabing-dagat kagabi.',
#    'Natuwa ako sa regalo mo.',
#    'Wala pa siya dito hanggang ngayon.',
#    'Pinili ko ang mas murang opsyon.',
#    'Kumalma ka muna at pag-usapan natin.',
#    'Mahilig kumain ng mangga ang pusa namin.',
#    'Bakit parang tumaba siya?',
#]

sentences2 = [
    'Naglaba ako ng mga damit kaninang umaga.',
    'Nagpa-deliver kami ng hapunan kagabi.',
    'Malamig ang hangin ngayong gabi.',
    'Naghugas ako ng pinggan pagkatapos kumain.',
    'Nakatulog ako nang maaga kahapon.',
    'Nagulat siya nang makita ako roon.',
    'Nagbasa ako ng libro habang umuulan.',
    'Masarap ang tinapay na binili ko.',
    'Nag-ayos ako ng kwarto bago umalis.',
    'Tumigil ang ulan pagkaraan ng ilang oras.',
    'Nagpaalam siya bago umuwi.',
    'Mabigat ang dala kong bag ngayon.',
    'Nagtanim kami ng mga halaman sa likod-bahay.',
    'Nagpakain ako ng pusa sa labas.',
    'Uminom siya ng tsaa bago matulog.',
    'Napakabilis ng oras ngayon.',
    'Nagsuot ako ng jacket dahil malamig.',
    'Nakita ko ang bagong signage sa kanto.',
    'Nagtanong ako kung bukas pa ang tindahan.',
    'Nakangiti siyang lumapit sa amin.',
    'Naghanda ako ng listahan ng mga bibilhin.',
    'Nakarating kami nang mas maaga kaysa sa inaasahan.',
    'Kumukunot ang noo niya habang nag-iisip.',
    'Nagpapraktis siya ng sayaw para sa event.',
    'Marami akong natutunan ngayong araw.',
    'Nawala ang ballpen ko sa opisina.',
    'Nagsimula na ang palabas nang dumating kami.',
    'Nagdulot ng trapik ang aksidente sa kanto.',
    'Nakakabingi ang ingay mula sa konstruksyon.',
    'Nagpakarga ako ng load sa tindahan.',
    'Masarap ang bagong timplang kape nila.',
    'Lumamig ang pagkain dahil natagalan ako.',
    'Nakita kong lumipad ang saranggola niya.',
    'Nagtawanan kami sa mga biro niya.',
    'Naalala ko ang meeting na nakalimutan ko.',
    'Nakasara ang pinto kaya kumatok ako.',
    'Mabigat ang ulan kaya nabasa ako.',
    'Nagsindi siya ng kandila nung brownout.',
    'Hiningi ko ang opinyon niya tungkol dito.',
    'Nahanap ko na ang nawawala kong relo.',
    'Nagpaalam ako na aalis nang sandali.',
    'Natapos ko ang trabaho nang mas mabilis.',
    'Nagpadala siya ng mensahe kaninang umaga.',
    'Napakasaya ng mga bata sa palaruan.',
    'Pinag-isipan ko muna bago sumagot.',
    'Binati ko siya sa kanyang tagumpay.',
    'Nakita ko ang buwan na napakaliwanag.',
    'Napansin kong tahimik ang paligid.',
    'Nagluto ako ng sopas dahil malamig ang panahon.',
    'Nanood ako ng sunrise sa may dalampasigan.',
]

translations = []

for sentence in sentences2:
    translation = translate(sentence)
    translations.append(translation[0])
    print(f'Tagalog: {sentence}')
    print(f'English: {translation[0]}')
    print()


Tagalog: Naglaba ako ng mga damit kaninang umaga.
English: I washed my clothes that morning.

Tagalog: Nagpa-deliver kami ng hapunan kagabi.
English: We were delivering dinner last night.

Tagalog: Malamig ang hangin ngayong gabi.
English: The wind is cold tonight.

Tagalog: Naghugas ako ng pinggan pagkatapos kumain.
English: I washed the dishes after eating.

Tagalog: Nakatulog ako nang maaga kahapon.
English: I went to bed early yesterday.

Tagalog: Nagulat siya nang makita ako roon.
English: He was surprised to see me there.

Tagalog: Nagbasa ako ng libro habang umuulan.
English: I was reading a book in the rain.

Tagalog: Masarap ang tinapay na binili ko.
English: The bread which I bought was good.

Tagalog: Nag-ayos ako ng kwarto bago umalis.
English: I cleaned the room before I left.

Tagalog: Tumigil ang ulan pagkaraan ng ilang oras.
English: After some hours the rain ceased.

Tagalog: Nagpaalam siya bago umuwi.
English: He said goodbye before he went home.

Tagalog: Mabigat ang

In [23]:
print("\n".join(translations))

I washed my clothes that morning.
We were delivering dinner last night.
The wind is cold tonight.
I washed the dishes after eating.
I went to bed early yesterday.
He was surprised to see me there.
I was reading a book in the rain.
The bread which I bought was good.
I cleaned the room before I left.
After some hours the rain ceased.
He said goodbye before he went home.
My bag is heavy today.
We planted plants in the back of the house.
I was feeding a cat outside.
And he drank a cup of tea before he slept.
The time is fast now.
I was wearing a jacket because it was cold.
I saw the new sign in the corner.
I asked if the store was still open.
He came to us smiling.
I have prepared a list of things to be bought.
We arrived earlier than expected.
And his face was smiling as he thought.
She practiced dancing for the event.
I learned a lot today.
My ballpen was lost in the office.
The show had already begun when we arrived.
The accident in the corner caused traffic.
The noise from the construc